In [22]:
import os
import cv2
import numpy as np
import csv
import glob as glob
from matplotlib import pyplot as plt
import xml.etree.ElementTree as ET
from lxml import etree
import pandas as pd

In [23]:
desired_img_format = '.jpg'

In [24]:
vid_folder = 'DATDATA'

datFiles = glob.glob(vid_folder+"/*.dat")
color = (255, 0, 0) 
  
# Line thickness of 2 px 
thickness = 2

In [19]:

source = "MW-R"
OUTPUT_DIR ="output"
INPUT_DIR = "input"
color = (255, 0, 0) 
  
# Line thickness of 2 px 
thickness = 2


def write_xml(xml_str, xml_path):
    # remove blank text before prettifying the xml
    parser = etree.XMLParser(remove_blank_text=True)
    root = etree.fromstring(xml_str, parser)
    # prettify
    xml_str = etree.tostring(root, pretty_print=True)
    # save to file
    with open(xml_path, 'wb') as temp_xml:
        temp_xml.write(xml_str)


def create_PASCAL_VOC_xml(xml_path, abs_path, folder_name, image_name, img_height, img_width, depth):
    # By: Jatin Kumar Mandav
    annotation = ET.Element('annotation')
    ET.SubElement(annotation, 'folder').text = folder_name
    ET.SubElement(annotation, 'filename').text = image_name
    ET.SubElement(annotation, 'path').text = abs_path
    source = ET.SubElement(annotation, 'source')
    ET.SubElement(source, 'database').text = 'Unknown'
    size = ET.SubElement(annotation, 'size')
    ET.SubElement(size, 'width').text = img_width
    ET.SubElement(size, 'height').text = img_height
    ET.SubElement(size, 'depth').text = depth
    ET.SubElement(annotation, 'segmented').text = '0'
    print(xml_path)
    xml_str = ET.tostring(annotation)
    write_xml(xml_str, xml_path)

def voc_format(class_name, point_1, point_2):
    # Order: class_name xmin ymin xmax ymax
    xmin, ymin = min(point_1[0], point_2[0]), min(point_1[1], point_2[1])
    xmax, ymax = max(point_1[0], point_2[0]), max(point_1[1], point_2[1])
    items = map(str, [class_name, xmin, ymin, xmax, ymax])
    return items
    
def append_bb(ann_path, line, extension):
    if '.txt' in extension:
        with open(ann_path, 'a') as myfile:
            myfile.write(line + '\n') # append line
    if '.xml' in extension:

        class_name, xmin, ymin, xmax, ymax = line

        tree = ET.parse(ann_path)
        annotation = tree.getroot()

        obj = ET.SubElement(annotation, 'object')
        ET.SubElement(obj, 'name').text = class_name
        ET.SubElement(obj, 'pose').text = 'Unspecified'
        ET.SubElement(obj, 'truncated').text = '0'
        ET.SubElement(obj, 'difficult').text = '0'

        bbox = ET.SubElement(obj, 'bndbox')
        ET.SubElement(bbox, 'xmin').text = xmin
        ET.SubElement(bbox, 'ymin').text = ymin
        ET.SubElement(bbox, 'xmax').text = xmax
        ET.SubElement(bbox, 'ymax').text = ymax

        xml_str = ET.tostring(annotation)
        write_xml(xml_str, ann_path)


def get_annotation_paths(img_path, annotation_formats):
    annotation_paths = []
    for ann_dir, ann_ext in annotation_formats.items():
        new_path = os.path.join(OUTPUT_DIR, ann_dir)
        new_path = img_path.replace(INPUT_DIR, new_path, 1)
        pre_path, img_ext = os.path.splitext(new_path)
        new_path = new_path.replace(img_ext, ann_ext, 1)
        annotation_paths.append(new_path)
    return annotation_paths

def save_bounding_box(annotation_paths, class_index, point_1, point_2, width, height):
    for ann_path in annotation_paths:
        # if '.txt' in ann_path:
        #     line = yolo_format("person", point_1, point_2, width, height)
        #     append_bb(ann_path, line, '.txt')
        print(ann_path)
        if '.xml' in ann_path:
            line = voc_format("person", point_1, point_2)
            append_bb(ann_path, line, '.xml')

annotation_formats = {'PASCAL_VOC' : '.xml', 'YOLO_darknet' : '.txt'}
annotation_formats = {'PASCAL_VOC' : '.xml'}


In [21]:
for datFile in datFiles:
    # read flash.dat to a list of lists
    datContent = [i.strip().split() for i in open(datFile).readlines()]
    videoFile = datFile[:-4]+".mp4"
    dataFrame = pd.DataFrame.from_records(datContent)
    dataFrame.columns = ["ID","xmin","ymin","xmax","ymax","frame","lost","occluded","generated","label","attributes","att","attt"]

    # Create a VideoCapture object
    cap = cv2.VideoCapture(videoFile)

    # Check if camera opened successfully
    if (cap.isOpened() == False): 
        print("Unable to read camera feed")

    # Default resolutions of the frame are obtained.The default resolutions are system dependent.
    # We convert the resolutions from float to integer.
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    output_file=videoFile[:-4]+"output.avi"
    # Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.
    out = cv2.VideoWriter(output_file,cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))
#     print(filename)
    file_path, file_extension = os.path.splitext(videoFile)
    file_extension = file_extension.replace('.', '_')
    file_path += file_extension
    video_name_ext = os.path.basename(file_path)
    if not os.path.exists(file_path):
        print(' Converting video to individual frames...')
        os.makedirs(file_path)
    i=0
    while(True):
        ret, frame = cap.read()
        print(i)
        if ret == True: 
            selectedbox=dataFrame[dataFrame["frame"]==str(i)]
        # Write the frame into the file 'output.avi'
            boxes = selectedbox
            print(boxes)
            frame_name =  '{}_{}{}'.format(video_name_ext, i, desired_img_format)
            frame_path = os.path.join(file_path, frame_name)
            print(frame_name+"helloooooooooooooo")
            cv2.imwrite(frame_path, frame)
            
            annotation_paths = get_annotation_paths(frame_path, annotation_formats)
            abs_path = os.path.abspath(frame_path)
            folder_name = os.path.dirname(frame_path)
            image_name = os.path.basename(frame_path)

            img_height, img_width, depth = (str(number) for number in frame.shape)
            for ann_path_ in annotation_paths:
                    if not os.path.isfile(ann_path_):
                        create_PASCAL_VOC_xml(ann_path_, abs_path, folder_name, image_name, img_height, img_width, depth)
            for index, row in boxes.iterrows():
                print(index)
                print(list(row))
                box = list(row[1:5])
                print(box)
                save_bounding_box(annotation_paths, 0, (int(box[0]),int(box[1])), (int(box[2]),int(box[3])), img_width, img_height)
                frame = cv2.rectangle(frame, (int(box[0]),int(box[1])), (int(box[2]),int(box[3])), color, thickness)

            # Display the resulting frame    
            cv2.imshow('frame',frame)
            i = i + 1
            out.write(frame)

        # Press Q on keyboard to stop recording
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        # Break the loop
        else:
            break  

    # When everything done, release the video capture and video write objects
    cap.release()
    out.release()

    # Closes all the frames
    cv2.destroyAllWindows() 

0
     ID xmin ymin xmax ymax frame lost occluded generated     label  \
0     0  489  114  502  125     0    1        0         0   "baris"   
502   1  456  125  485  162     0    1        0         0  "ismail"   
1004  2  448  113  512  167     0    1        0         0    "alex"   

     attributes   att  attt  
0          None  None  None  
502        None  None  None  
1004       None  None  None  
top-1_mp4_0.jpghelloooooooooooooo
0
['0', '489', '114', '502', '125', '0', '1', '0', '0', '"baris"', None, None, None]
['489', '114', '502', '125']
DATDATA\top-1_mp4\top-1_mp4_0.xml
502
['1', '456', '125', '485', '162', '0', '1', '0', '0', '"ismail"', None, None, None]
['456', '125', '485', '162']
DATDATA\top-1_mp4\top-1_mp4_0.xml
1004
['2', '448', '113', '512', '167', '0', '1', '0', '0', '"alex"', None, None, None]
['448', '113', '512', '167']
DATDATA\top-1_mp4\top-1_mp4_0.xml
1
     ID xmin ymin xmax ymax frame lost occluded generated     label  \
1     0  489  114  502  125     1    

1014
['2', '448', '113', '512', '167', '10', '1', '0', '1', '"alex"', None, None, None]
['448', '113', '512', '167']
DATDATA\top-1_mp4\top-1_mp4_10.xml
11
     ID xmin ymin xmax ymax frame lost occluded generated     label  \
11    0  489  114  502  125    11    1        0         1   "baris"   
513   1  456  123  487  162    11    1        0         1  "ismail"   
1015  2  448  113  512  167    11    1        0         1    "alex"   

     attributes   att  attt  
11         None  None  None  
513        None  None  None  
1015       None  None  None  
top-1_mp4_11.jpghelloooooooooooooo
DATDATA\top-1_mp4\top-1_mp4_11.xml
11
['0', '489', '114', '502', '125', '11', '1', '0', '1', '"baris"', None, None, None]
['489', '114', '502', '125']
DATDATA\top-1_mp4\top-1_mp4_11.xml
513
['1', '456', '123', '487', '162', '11', '1', '0', '1', '"ismail"', None, None, None]
['456', '123', '487', '162']
DATDATA\top-1_mp4\top-1_mp4_11.xml
1015
['2', '448', '113', '512', '167', '11', '1', '0', '1', '"alex

DATDATA\top-1_mp4\top-1_mp4_21.xml
21
['0', '489', '114', '502', '125', '21', '1', '0', '1', '"baris"', None, None, None]
['489', '114', '502', '125']
DATDATA\top-1_mp4\top-1_mp4_21.xml
523
['1', '456', '122', '489', '162', '21', '1', '0', '1', '"ismail"', None, None, None]
['456', '122', '489', '162']
DATDATA\top-1_mp4\top-1_mp4_21.xml
1025
['2', '448', '113', '512', '167', '21', '1', '0', '1', '"alex"', None, None, None]
['448', '113', '512', '167']
DATDATA\top-1_mp4\top-1_mp4_21.xml
22
     ID xmin ymin xmax ymax frame lost occluded generated     label  \
22    0  489  114  502  125    22    1        0         1   "baris"   
524   1  456  122  489  162    22    1        0         1  "ismail"   
1026  2  448  113  512  167    22    1        0         1    "alex"   

     attributes   att  attt  
22         None  None  None  
524        None  None  None  
1026       None  None  None  
top-1_mp4_22.jpghelloooooooooooooo
DATDATA\top-1_mp4\top-1_mp4_22.xml
22
['0', '489', '114', '502', '

     ID xmin ymin xmax ymax frame lost occluded generated     label  \
33    0  489  114  502  125    33    1        0         1   "baris"   
535   1  456  121  491  162    33    1        0         1  "ismail"   
1037  2  448  113  512  167    33    1        0         1    "alex"   

     attributes   att  attt  
33         None  None  None  
535        None  None  None  
1037       None  None  None  
top-1_mp4_33.jpghelloooooooooooooo
DATDATA\top-1_mp4\top-1_mp4_33.xml
33
['0', '489', '114', '502', '125', '33', '1', '0', '1', '"baris"', None, None, None]
['489', '114', '502', '125']
DATDATA\top-1_mp4\top-1_mp4_33.xml
535
['1', '456', '121', '491', '162', '33', '1', '0', '1', '"ismail"', None, None, None]
['456', '121', '491', '162']
DATDATA\top-1_mp4\top-1_mp4_33.xml
1037
['2', '448', '113', '512', '167', '33', '1', '0', '1', '"alex"', None, None, None]
['448', '113', '512', '167']
DATDATA\top-1_mp4\top-1_mp4_33.xml
34
     ID xmin ymin xmax ymax frame lost occluded generated     labe

43
     ID xmin ymin xmax ymax frame lost occluded generated     label  \
43    0  489  114  502  125    43    1        0         1   "baris"   
545   1  456  120  493  162    43    1        0         1  "ismail"   
1047  2  422  125  500  182    43    0        0         1    "alex"   

     attributes   att  attt  
43         None  None  None  
545        None  None  None  
1047  "walking"  None  None  
top-1_mp4_43.jpghelloooooooooooooo
DATDATA\top-1_mp4\top-1_mp4_43.xml
43
['0', '489', '114', '502', '125', '43', '1', '0', '1', '"baris"', None, None, None]
['489', '114', '502', '125']
DATDATA\top-1_mp4\top-1_mp4_43.xml
545
['1', '456', '120', '493', '162', '43', '1', '0', '1', '"ismail"', None, None, None]
['456', '120', '493', '162']
DATDATA\top-1_mp4\top-1_mp4_43.xml
1047
['2', '422', '125', '500', '182', '43', '0', '0', '1', '"alex"', '"walking"', None, None]
['422', '125', '500', '182']
DATDATA\top-1_mp4\top-1_mp4_43.xml
44
     ID xmin ymin xmax ymax frame lost occluded generate

...

1457
['2', '477', '112', '515', '140', '453', '1', '1', '1', '"alex"', '"walking"', None, None]
['477', '112', '515', '140']
DATDATA\top-1_mp4\top-1_mp4_453.xml
454
     ID xmin ymin xmax ymax frame lost occluded generated     label  \
454   0  381  118  478  196   454    0        0         1   "baris"   
956   1  458  112  514  163   454    1        0         1  "ismail"   
1458  2  477  112  515  140   454    1        1         1    "alex"   

     attributes   att  attt  
454   "walking"  None  None  
956   "walking"  None  None  
1458  "walking"  None  None  
top-1_mp4_454.jpghelloooooooooooooo
DATDATA\top-1_mp4\top-1_mp4_454.xml
454
['0', '381', '118', '478', '196', '454', '0', '0', '1', '"baris"', '"walking"', None, None]
['381', '118', '478', '196']
DATDATA\top-1_mp4\top-1_mp4_454.xml
956
['1', '458', '112', '514', '163', '454', '1', '0', '1', '"ismail"', '"walking"', None, None]
['458', '112', '514', '163']
DATDATA\top-1_mp4\top-1_mp4_454.xml
1458
['2', '477', '112', '515', '14

DATDATA\top-1_mp4\top-1_mp4_466.xml
466
['0', '446', '115', '511', '167', '466', '0', '1', '0', '"baris"', '"walking"', None, None]
['446', '115', '511', '167']
DATDATA\top-1_mp4\top-1_mp4_466.xml
968
['1', '458', '112', '514', '163', '466', '1', '0', '1', '"ismail"', '"walking"', None, None]
['458', '112', '514', '163']
DATDATA\top-1_mp4\top-1_mp4_466.xml
1470
['2', '477', '112', '515', '140', '466', '1', '1', '1', '"alex"', '"walking"', None, None]
['477', '112', '515', '140']
DATDATA\top-1_mp4\top-1_mp4_466.xml
467
     ID xmin ymin xmax ymax frame lost occluded generated     label  \
467   0  450  115  511  162   467    0        1         1   "baris"   
969   1  458  112  514  163   467    1        0         1  "ismail"   
1471  2  477  112  515  140   467    1        1         1    "alex"   

     attributes   att  attt  
467   "walking"  None  None  
969   "walking"  None  None  
1471  "walking"  None  None  
top-1_mp4_467.jpghelloooooooooooooo
DATDATA\top-1_mp4\top-1_mp4_467.xml

DATDATA\top-1_mp4\top-1_mp4_478.xml
478
['0', '469', '115', '511', '145', '478', '1', '1', '1', '"baris"', '"walking"', None, None]
['469', '115', '511', '145']
DATDATA\top-1_mp4\top-1_mp4_478.xml
980
['1', '458', '112', '514', '163', '478', '1', '0', '1', '"ismail"', '"walking"', None, None]
['458', '112', '514', '163']
DATDATA\top-1_mp4\top-1_mp4_478.xml
1482
['2', '477', '112', '515', '140', '478', '1', '1', '1', '"alex"', '"walking"', None, None]
['477', '112', '515', '140']
DATDATA\top-1_mp4\top-1_mp4_478.xml
479
     ID xmin ymin xmax ymax frame lost occluded generated     label  \
479   0  469  115  511  145   479    1        1         1   "baris"   
981   1  458  112  514  163   479    1        0         1  "ismail"   
1483  2  477  112  515  140   479    1        1         1    "alex"   

     attributes   att  attt  
479   "walking"  None  None  
981   "walking"  None  None  
1483  "walking"  None  None  
top-1_mp4_479.jpghelloooooooooooooo
DATDATA\top-1_mp4\top-1_mp4_479.xml

1493
['2', '477', '112', '515', '140', '489', '1', '1', '1', '"alex"', '"walking"', None, None]
['477', '112', '515', '140']
DATDATA\top-1_mp4\top-1_mp4_489.xml
490
     ID xmin ymin xmax ymax frame lost occluded generated     label  \
490   0  469  115  511  145   490    1        1         1   "baris"   
992   1  458  112  514  163   490    1        0         1  "ismail"   
1494  2  477  112  515  140   490    1        1         1    "alex"   

     attributes   att  attt  
490   "walking"  None  None  
992   "walking"  None  None  
1494  "walking"  None  None  
top-1_mp4_490.jpghelloooooooooooooo
DATDATA\top-1_mp4\top-1_mp4_490.xml
490
['0', '469', '115', '511', '145', '490', '1', '1', '1', '"baris"', '"walking"', None, None]
['469', '115', '511', '145']
DATDATA\top-1_mp4\top-1_mp4_490.xml
992
['1', '458', '112', '514', '163', '490', '1', '0', '1', '"ismail"', '"walking"', None, None]
['458', '112', '514', '163']
DATDATA\top-1_mp4\top-1_mp4_490.xml
1494
['2', '477', '112', '515', '14

1504
['2', '477', '112', '515', '140', '500', '1', '1', '1', '"alex"', '"walking"', None, None]
['477', '112', '515', '140']
DATDATA\top-1_mp4\top-1_mp4_500.xml
501
     ID xmin ymin xmax ymax frame lost occluded generated     label  \
501   0  469  115  511  145   501    1        1         0   "baris"   
1003  1  458  112  514  163   501    1        0         0  "ismail"   
1505  2  477  112  515  140   501    1        1         0    "alex"   

     attributes   att  attt  
501   "walking"  None  None  
1003  "walking"  None  None  
1505  "walking"  None  None  
top-1_mp4_501.jpghelloooooooooooooo
DATDATA\top-1_mp4\top-1_mp4_501.xml
501
['0', '469', '115', '511', '145', '501', '1', '1', '0', '"baris"', '"walking"', None, None]
['469', '115', '511', '145']
DATDATA\top-1_mp4\top-1_mp4_501.xml
1003
['1', '458', '112', '514', '163', '501', '1', '0', '0', '"ismail"', '"walking"', None, None]
['458', '112', '514', '163']
DATDATA\top-1_mp4\top-1_mp4_501.xml
1505
['2', '477', '112', '515', '1

In [58]:
dataFrame[dataFrame["frame"]=="0"]

,ID,xmin,ymin,xmax,ymax,frame,lost,occluded,generated,label,attributes,att,attt
0,0,453,137,510,179,0,1,0,0,"""person""",None,None,None


In [26]:
top_ ="top-1.dat"

In [31]:
# read flash.dat to a list of lists
datContent = [i.strip().split() for i in open(top_).readlines()]

FileNotFoundError: [Errno 2] No such file or directory: 'top-1.dat'

In [28]:
len(datContent[0])

10

In [29]:
len(["ID","xmin","ymin","xmax","ymax","frame","lost","occluded","generated","label","attributes"])

11

In [22]:
# datContent
df = pd.DataFrame.from_records(datContent)
df.columns = ["ID","xmin","ymin","xmax","ymax","frame","lost","occluded","generated","label","attributes","att","attt"]
#     Track ID. All rows with the same ID belong to the same path.
#     xmin. The top left x-coordinate of the bounding box.
#     ymin. The top left y-coordinate of the bounding box.
#     xmax. The bottom right x-coordinate of the bounding box.
#     ymax. The bottom right y-coordinate of the bounding box.
#     frame. The frame that this annotation represents.
#     lost. If 1, the annotation is outside of the view screen.
#     occluded. If 1, the annotation is occluded.
#     generated. If 1, the annotation was automatically interpolated.
#     label. The label for this annotation, enclosed in quotation marks.
#     attributes. Each column after this is an attribute.
#     attributes. there can be multiple attributes.

In [42]:
df.to_csv("top.csv")

In [24]:

# # write it as a new CSV file
# with open("./flash.csv", "wb") as f:
#     writer = csv.writer(f)
#     writer.writerows(datContent)

In [1]:
datContent

NameError: name 'datContent' is not defined